In [ ]:
## Start from this Cell
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 68.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 78.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 45.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.39.0-py3-none-any.whl size=25319738 sha256=0e65f876179681d85f471f4fbe0065970dd391fe3c1124f0feb635289620b82b
  Stored in directory: /root/.cache/pip/wheels/98/fd/0a/ef6252223f3d2

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import ktrain
from ktrain import text
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/usman ms thesis/Dataset/Training/6_data.csv')

In [ ]:
# del df['id']

In [ ]:
!pip install transformers

In [ ]:
df

,Unnamed: 0,text,label,language
0,102554,Так то делают такие операции...,0.0,6
1,102555,"Реальная история, тигра тогда ещё Малахов к се...",0.0,6
2,102556,"Нас отделяет 2,5к км) Так что остаётся только ...",0.0,6
3,102557,"И вот по последнему пункту статьи у тс, после ...",0.0,6
4,102558,жизнь вообще сложная штука,0.0,6
...,...,...,...,...
14503,117057,"а вот осквернение, а именно глумление над прин...",0.0,6
14504,117058,Если все из перечисленного исправляется химчис...,0.0,6
14505,117059,"ЖЕНЩИНА это ВЕНЕЦ ТВОРЕНИЯ, помните это ваньки...",1.0,6
14506,117060,Совершенно верно. Крышняк едет ого-го.,1.0,6


In [ ]:
!pip install langdetect


In [ ]:
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Unknown'

df['DetectedLanguage'] = df['text'].apply(detect_language)

In [ ]:
del df['Unnamed: 0']

In [ ]:
df['DetectedLanguage'].value_counts()

ru         13860
uk           271
bg           207
mk           159
en             4
hr             1
id             1
Unknown        1
sk             1
so             1
de             1
ca             1
Name: DetectedLanguage, dtype: int64

In [ ]:
condition = df['DetectedLanguage'] =='ru'
df_train = df[condition]

In [ ]:
df_train

,text,label,language,DetectedLanguage
0,Так то делают такие операции...,0.0,6,ru
1,"Реальная история, тигра тогда ещё Малахов к се...",0.0,6,ru
2,"Нас отделяет 2,5к км) Так что остаётся только ...",0.0,6,ru
3,"И вот по последнему пункту статьи у тс, после ...",0.0,6,ru
4,жизнь вообще сложная штука,0.0,6,ru
...,...,...,...,...
14502,"И ещё один, какая же русня дегенераты пиздец п...",1.0,6,ru
14503,"а вот осквернение, а именно глумление над прин...",0.0,6,ru
14504,Если все из перечисленного исправляется химчис...,0.0,6,ru
14505,"ЖЕНЩИНА это ВЕНЕЦ ТВОРЕНИЯ, помните это ваньки...",1.0,6,ru


In [ ]:
X=df_train['text']
y=df_train['label']

In [ ]:
from sklearn.model_selection import train_test_split


# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.7, random_state=42, shuffle=True, stratify=y)
X_test,X_val, y_test,y_val = train_test_split(X_rem, y_rem, test_size=1/3, random_state=42, shuffle=True, stratify=y_rem)

print(X_train.shape), print(y_train.shape)
print(X_val.shape), print(y_val.shape)
print(X_test.shape)

(9702,)
(9702,)
(1386,)
(1386,)
(2772,)


In [ ]:
x =y_train.value_counts()
label_0 = round(x[0]/sum(x)*100,2)
label_1 = round(x[1]/sum(x)*100,2)
print("Label_0 = {} \nLabel_1 = {}".format(label_0,label_1))

Label_0 = 66.94 
Label_1 = 33.06


In [ ]:
MODEL_NAME = 'bert-base-cased'

t = text.Transformer(MODEL_NAME, maxlen=400, classes=np.array([0,1]))
trn = t.preprocess_train(np.array(X_train),np.array(y_train))
val = t.preprocess_test(np.array(X_val), np.array(y_val))
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=12)

/usr/local/lib/python3.10/dist-packages/ktrain/text/preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

preprocessing train...
language: ru
train sequence lengths:
	mean : 29
	95percentile : 86
	99percentile : 189


/usr/local/lib/python3.10/dist-packages/ktrain/utils.py:768: UserWarning: class_names implies classification but targets array contains float(s) instead of integers or strings
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: ru
test sequence lengths:
	mean : 29
	95percentile : 92
	99percentile : 180


In [ ]:
learner.fit_onecycle(1e-5, 3)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/3
809/809 [==============================] - 944s 1s/step - loss: 0.5948 - accuracy: 0.6849 - val_loss: 0.5041 - val_accuracy: 0.7626
Epoch 2/3
809/809 [==============================] - 913s 1s/step - loss: 0.4449 - accuracy: 0.7922 - val_loss: 0.3773 - val_accuracy: 0.8341
Epoch 3/3
809/809 [==============================] - 954s 1s/step - loss: 0.3213 - accuracy: 0.8588 - val_loss: 0.3494 - val_accuracy: 0.8492


In [ ]:
pred = ktrain.get_predictor(learner.model,preproc = t)

In [ ]:
X_test = [(text, '') for text in X_test]


In [ ]:
predictor_1=pred.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
print(confusion_matrix(y_test,predictor_1))

[[1725  131]
 [ 220  696]]


In [ ]:
print(classification_report(y_test,predictor_1))
print(accuracy_score(y_test, predictor_1))

              precision    recall  f1-score   support

         0.0       0.89      0.93      0.91      1856
         1.0       0.84      0.76      0.80       916

    accuracy                           0.87      2772
   macro avg       0.86      0.84      0.85      2772
weighted avg       0.87      0.87      0.87      2772

0.8733766233766234
